[ЧМ по ЧГК](http://worldchamp.chgk.info/) 2017 года пройдёт по довольно странным правилам: чемпион будет определяться только на последних 30 вопросах (из 120) (в регламенте есть и бо́льшие странности, но про них сегодня не будет ничего). Давайте посмотрим, что получится, если повыдёргивать победителей каких-то двух туров на предыдущих ЧР и ЧМ.

Итог этого упражнения — [красивый график в Tableau](https://public.tableau.com/views/30-questions/30?:embed=true&:display_count=yes), а также два CSV-файла в этом репозитории. Между кодом есть некоторые пояснения, а ещё они есть [в статье на chgk.gg](https://chgk.gg/chm-2017-30-questions/).

Скучные импорты и определения, листайте дальше.

In [1]:
import requests
import json
from typing import Dict, NamedTuple, List, Tuple
from itertools import chain
from collections import Counter
import csv

In [2]:
class Team:
    def __init__(self, team: Dict, tour_len=15):
        self.name = team['current_name']
        self.position = float(team['position'])
        self.tours_count = len(team['mask']) // tour_len
        tours_str = (team['mask'][i*tour_len:(i+1)*tour_len] 
                     for i in range(self.tours_count))
        self.tours = [sum(int(q) for q in tour) for tour in tours_str]
        
    @property
    def sum(self):
        return sum(self.tours)
    
    @property
    def pairs(self):
        return {f'{i + 1} и {j + 1}': self.tours[i] + self.tours[j]
                 for i in range(self.tours_count)
                for j in range(i + 1, self.tours_count)}
    
    @property
    def full_name(self):
        return f'{int(self.position)}. {self.name} ({self.sum})'
        
        
class PairResult(NamedTuple):
    tournament_name: str
    tours: str
    teams: str
    result: int

class WinPercentage(NamedTuple):
    tournament_name: str
    team: str
    percentage: float

Тут можно добавить ID своего топового турнира и посмотреть, что было бы (понятно, что интересно взглянуть на финальные этапы ЧМ, но туры по 15 вопросов были только в 2013 году).

In [3]:
def load_data(id: int) -> Tuple[Dict]:
    metadata = requests.get(f'http://rating.chgk.info/api/tournaments/{id}.json').json()[0]
    results = requests.get(f'http://rating.chgk.info/api/tournaments/{id}/list.json').json()
    return metadata, results


tournaments = [4247, 3825, 3099, 2813, 2117, 1983, #ЧР
               4030, 3495, 2958, 2481, 2422, 2086] #ЧМ
data = [load_data(id) for id in tournaments]

Забрали данные из рейтинга, теперь можно составить все пары туров и сразу же найти лучшие команды в этих парах.

In [4]:
def process_tournament(metadata: Dict, results: Dict) -> List[PairResult]:
    raw_name = metadata['name']
    if 'России' in raw_name:
        tournament_name = f'ЧР {metadata["date_end"][:4]}'
    elif 'Финал' in raw_name:
        tournament_name = f'Финал ЧМ {metadata["date_end"][:4]}'
    else:
        tournament_name = f'Отбор ЧМ {metadata["date_end"][:4]}'
    teams = [Team(team) for team in results]
    keys = teams[0].pairs.keys()
    return [find_max_result(tournament_name, key, teams) for key in keys]

def find_max_result(tournament_name: str, key: str, teams: List[Team]) -> PairResult:
    best_teams = []
    best_result = 0
    for team in teams:
        team_result = team.pairs[key]
        if team_result == best_result:
            best_teams += [team.full_name]
        if team_result > best_result:
            best_result = team_result
            best_teams = [team.full_name]
            
    return PairResult(tournament_name, key, best_teams, best_result)

results_with_pairs = [process_tournament(metadata, results) 
                      for metadata, results in data]

Теперь посчитаем, какова вероятность победы команды при игре по этим правилам. Например, в отборе ЧМ четыре тура, а разных пар — шесть (обязательно проверьте!), поэтому если ваша команда выиграла три пары, то вероятность, что мы вытащим удачную для вас пару, — 0,5.

`percentages` — это список списков, для каждого турнира — свой, тут мы для экономии места покажем только проценты последнего ЧР.

In [5]:
def count_percentages(tournament: List[PairResult]) -> List[WinPercentage]:
    winners = list(chain.from_iterable(result.teams for result in tournament))
    counter = Counter(winners)
    return [WinPercentage(tournament[0].tournament_name, team,
                         round(100 * wins / len(tournament), 1))
            for team, wins in counter.items()]


percentages = [count_percentages(r) for r in results_with_pairs]
percentages[0]

[WinPercentage(tournament_name='ЧР 2017', team='2. Рабочее название (58)', percentage=20.0),
 WinPercentage(tournament_name='ЧР 2017', team='1. Борский корабел (59)', percentage=46.7),
 WinPercentage(tournament_name='ЧР 2017', team='3. Команда Губанова (57)', percentage=26.7),
 WinPercentage(tournament_name='ЧР 2017', team='4. Сборная Кирибати (51)', percentage=6.7),
 WinPercentage(tournament_name='ЧР 2017', team='5. Мираж (50)', percentage=20.0),
 WinPercentage(tournament_name='ЧР 2017', team='9. ТПРУНЯ (47)', percentage=6.7)]

Скучный экспорт в CSV.

In [6]:
with open('pct.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    pct_list = list(chain.from_iterable(percentages))
    writer.writerow(['Турнир', 'Команда', 'Процент побед'])
    writer.writerows(pct_list)
    
with open('pairs.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    writer.writerow(['Турнир', 'Пара туров', 'Команды', 'Сумма'])
    pairs = chain.from_iterable(results_with_pairs)
    for pair in pairs:
        writer.writerow([pair.tournament_name, pair.tours, 
                        ' и '.join(sorted(pair.teams)), pair.result])